In [9]:
import os, sys
import base64
import matplotlib.pyplot as plt
import io
import pandas as pd
from PIL import Image

target_cwd = '/workspace/repo/cccrawl'
sys.path.append(target_cwd)
sys.path.append('/workspace/repo/cccrawl/OmniParser')
os.chdir(target_cwd)
print(os.getcwd())

from bing_search.models import OmniParserModel

/workspace/repo/cccrawl


In [10]:
model = OmniParserModel()

In [11]:

plt.figure(figsize=(15,15))

image_root_path ='/workspace/repo/cccrawl/OmniParser3/imgs'

meta_data = []

for image_path in os.listdir(image_root_path):
	image_path = os.path.join(image_root_path, image_path)
	print(image_path)
	results = model.process_image(image_path)
	
	dino_labled_img = results['dino_labled_img']
	label_coordinates = results['label_coordinates']
	parsed_content_list = results['parsed_content_list']
	w, h = results['image_size']
	
	# save dino_labled_img to file

	image = Image.open(io.BytesIO(base64.b64decode(dino_labled_img)))
	
	image.save('output.png')

	plt.axis('off')
	plt.imshow(image)

	print(len(parsed_content_list))

	def convert(parsed_content_list):
		results = []
		for item in parsed_content_list:
			result = {
				'img_filename': image_path,
				'bbox': [
					int(item['bbox'][0] * w),
					int(item['bbox'][1] * h),
					int(item['bbox'][2] * w),
					int(item['bbox'][3] * h)
				],
				'instruction': item['content'],
				'data_app': "data_app",
				'data_source': item['type'],
				'resolution': (w,h)
			}
			results.append(result)
		return results

	results = convert(parsed_content_list)

	meta_data.extend(results)

	break
	

# label_coordinates is a list of dictionaries with the following format:
# {
# 'type': 'text',
# 'bbox': [0.17,0.01,0.21,0.03], # [v[0]/w, v[1]/h, v[2]/w, v[3]/h]
# 'interactivity': False,
# 'content': 'storage'
# },

# i need to convert the bbox to the format that is used in the dataset
# {
#     "img_filename": "pc_pl3ry1.png",
#     "bbox": [
#       x1,
#       y2,
#       x2,
#       y2
#     ],
#     "instruction": "update all available apps",
#     "data_app": "Paint", 
#     "data_source": "xxx", 
#     "resolution": 
#   },

/workspace/repo/cccrawl/OmniParser3/imgs/windows_multitab.png

0: 864x1280 100 icons, 12.2ms
Speed: 5.9ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 864, 1280)
len(filtered_boxes): 109 61


ValueError: Attention mask should be of size (48, 1, 10, 10), but is torch.Size([48, 1, 5, 5])

<Figure size 1500x1500 with 0 Axes>

In [ ]:
import json

from bing_search.util import generativedatasets_get_dir, generativedatasets_path_to_info

org_data_root = "/mnt/vground/generativedatasets/generativedatasets/gold_en/collection=base/data/office/"

dirs, meta = generativedatasets_get_dir(org_data_root)

save_data_root = '/mnt/vground/generativedatasets/generativedatasets_processed/gold_en/collection=base/data/office/'

meta_data = []

for dir in dirs:
	image_paths = [os.path.join(dir, file_name) for file_name in os.listdir(dir) if file_name.endswith('.png')]
	dir_info = generativedatasets_path_to_info(dir)
	app = dir_info['app']

	for image_path in image_paths:
		print(image_path)
		results = model.process_image(image_path)
		dino_labled_img = results['dino_labled_img']
		parsed_content_list = results['parsed_content_list']
		w, h = results['image_size']
		# save dino_labled_img to file
		image = Image.open(io.BytesIO(base64.b64decode(dino_labled_img)))

		image_save_path = image_path.replace(org_data_root, save_data_root).replace('.png', '_parsed.png')

		save_dir = os.path.dirname(image_save_path)

		os.makedirs(save_dir, exist_ok=True)

		image.save(image_save_path)
	
		print(len(parsed_content_list))

		def convert(parsed_content_list,image_path, app, w, h):
			results = []
			for item in parsed_content_list:
				result = {
					'img_filename': image_path.replace(org_data_root, ''),
					'bbox': [
						int(item['bbox'][0] * w),
						int(item['bbox'][1] * h),
						int(item['bbox'][2] * w),
						int(item['bbox'][3] * h)
					],
					'instruction': item['content'],
					'data_app': app,
					'data_source': item['type'],
					'resolution': (w,h)
				}
				results.append(result)
			return results
		
		results = convert(parsed_content_list, image_path, app, w, h)

		# save results to json
		json_save_path = image_save_path.replace('.png', '_parsed.json')
		with open(json_save_path, 'w') as f:
			json.dump(results, f)

		print(f'save image to {image_save_path}, json to {json_save_path}')

		meta_data.extend(results)
		break

df = pd.DataFrame(meta_data)
df.to_csv(os.path.join(save_data_root, 'metadata.csv'), index=False)


In [3]:
meta_file  = '/workspace/repo/cccrawl/generativedatasets_processed/metadata.csv'

df = pd.read_csv(meta_file)

df.head()

,img_filename,bbox,instruction,data_app,data_source,resolution
0,resolution=1920x1080/language=en-us/theme=aero...,"[18, 6, 45, 31]",H,excel,text,"(1920, 1080)"
1,resolution=1920x1080/language=en-us/theme=aero...,"[275, 3, 653, 30]",0B14B7FD3E94D0E00C4F62D539BCA849 [Compatibilit...,excel,text,"(1920, 1080)"
2,resolution=1920x1080/language=en-us/theme=aero...,"[798, 3, 876, 31]",8Search,excel,text,"(1920, 1080)"
3,resolution=1920x1080/language=en-us/theme=aero...,"[1667, 3, 1732, 31]",Sign in,excel,text,"(1920, 1080)"
4,resolution=1920x1080/language=en-us/theme=aero...,"[64, 40, 110, 63]",Home,excel,text,"(1920, 1080)"
